In [1]:
# Python Script 2: Fractionation
# 1) Register 288 new samples
# 2) Register 3 Fraction Plate maps

import pandas as pd
import requests
import json
from datetime import date
import numpy as np


# hardcoded variables

CELLULAR_FRACTION = ['Nucleoplasm', 'Chromatin', 'Insoluble'] # iterate through each to get the full 3x fractionation
TX_TIME = 4
TX_TIME_UNIT = "hours"
VOL = 125
HARVEST_PROTOCOL_ID = "PR003-V1"
FRX_PROTOCOL_ID = "PR004-V1"
PREP_BY = "Bodhi Hueffmeier"

#PLATE_BARCODE = 
#PLATE_NAME = 
#WELL_POSITION = 

In [ ]:
# 1) Register 288 new samples
# i.e. 3x 96 samples

def cell_fraction_rows(name, cell_frx):
    rows = [
    "AUTOMATIC", # Registry ID - don't do anything with   Registry ID,
    name, # Name (temporary field)                   Name,
    cell_frx, # Cellular Fraction
    TX_TIME, # Post-Treatment Time Point             Post-Treatment Time Point,
    TX_TIME_UNIT, # Time Point Unit                  Time Point Unit
    VOL, # Volume (uL)                               Volume (uL),
    "", # Protein Concentration (ug/uL)              Protein Concentration (ug/uL),
    HARVEST_PROTOCOL_ID, # Harvest Protocol ID       Harvest Protocol ID,
    FRX_PROTOCOL_ID, # Fractionation Protocol ID     Fractionation Protocol ID,
    PREP_BY, # Prepared By                           Prepared By,
    str(date.today()), # Preparation Date            Preparation Date,
    "", # Record Creator                             Record Creator,
    "" # Storage Location                           Storage Location
    #PLATE_BARCODE, # Plate Barcode                   Plate Barcode
    #PLATE_NAME, # Plate Name
    #WELL_POSITION # Well Position
    ]

    return rows

* how should we handle the "extra" plate-location information? 